In [3]:
import pandas as pd
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import re
import nltk

In [4]:
artists = pd.read_csv("artists-data.csv")
lyrics = pd.read_csv("lyrics-data.csv")


In [5]:
artists

,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/
...,...,...,...,...,...
4163,Miriam Makeba,World Music; Black Music; Blues,17.0,0.0,/miriam-makeba/
4164,Freddie Aguilar,World Music,61.0,0.0,/freddie-aguilar/
4165,Amadou & Mariam,World Music,14.0,0.0,/amadou-mariam/
4166,Magic System,World Music; Gospel/Religioso,16.0,0.0,/magic-system/


In [13]:
unique_genres = set()
for genres in artists['Genres'].str.split(';').dropna():
    unique_genres.update(genres)
# Convert the set to a list if needed
unique_genres_list = [genre for genre in unique_genres if ' ' not in genre]
unique_genres_list

['Gótico',
 'Forró',
 'Kizomba',
 'Metal',
 'Psicodelia',
 'Industrial',
 'Pop/Punk',
 'Pós-Punk',
 'Reggae',
 'Power-Pop',
 'Disco',
 'Urban',
 'Samba',
 'Tecnopop',
 'Grunge',
 'Post-Rock',
 'Instrumental',
 'Fado',
 'Ska',
 'Clássico',
 'Infantil',
 'Regional',
 'COLETÂNEA',
 'Hardcore',
 'Progressivo',
 'Pagode',
 'Country',
 'Reggaeton',
 'Funk',
 'Gospel/Religioso',
 'House',
 'Indie',
 'Trip-Hop',
 'R&B',
 'MPB',
 'Rock',
 'Piseiro',
 'Blues',
 'Trance',
 'Electronica',
 'Axé',
 'J-Pop/J-Rock',
 'Emocore',
 'Jazz',
 'Folk',
 'K-Pop/K-Rock',
 'Rap',
 'Dance',
 'Pop/Rock',
 'Romântico',
 'Chillout',
 'Rockabilly',
 'Lo-fi',
 'Pop',
 'Sertanejo',
 'Trap']

In [14]:
len(unique_genres_list)

56

In [5]:
lyrics

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [40]:
lyrics_en = lyrics[lyrics['language'] == 'en']
lyrics_en = lyrics_en.reset_index(drop=True)
lyrics_en

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en
1,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en
2,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en
3,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en
4,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en
...,...,...,...,...,...
191809,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
191810,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
191811,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
191812,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [33]:
lyrics_en[lyrics_en['Lyric'].isna()]


,ALink,SName,SLink,Lyric,language,Lyrics_Clean,Sname


In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/carolinacraus/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
lyrics_en.loc[:, 'Lyrics_Clean'] = lyrics_en['Lyric'].apply(basic_text_cleaning)

In [42]:
lyrics_en 


,ALink,SName,SLink,Lyric,language,Lyrics_Clean
0,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en,"[i, feel, so, unsure, as, i, take, your, hand,..."
1,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en,"[dont, let, them, fool, ya, or, even, try, to,..."
2,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en,"[baby, lets, cruise, away, from, here, dont, b..."
3,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en,"[know, it, sounds, funny, but, i, just, cant, ..."
4,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en,"[youve, got, that, look, again, the, one, i, h..."
...,...,...,...,...,...,...
191809,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en,"[chorus, here, we, stand, waiting, on, the, pl..."
191810,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en,"[i, nearly, disappeared, into, the, mouth, of,..."
191811,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en,"[amambuka, amambuka, azothengisa, izwe, lakith..."
191812,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en,"[sweat, in, the, heat, for, days, on, end, wai..."


In [37]:
for index, row in lyrics_en.iterrows():
    lyrics_list = row['Lyrics_Clean']
    
    # Check if 'chorus' or 'verse' is in the lyrics list
    if 'chorus' in lyrics_list or 'verse' in lyrics_list:
        print(f"Row {index}: Found 'chorus' or 'verse'")

Row 3254: Found 'chorus' or 'verse'
Row 4761: Found 'chorus' or 'verse'
Row 4766: Found 'chorus' or 'verse'
Row 4782: Found 'chorus' or 'verse'
Row 4785: Found 'chorus' or 'verse'
Row 4796: Found 'chorus' or 'verse'
Row 4799: Found 'chorus' or 'verse'
Row 4802: Found 'chorus' or 'verse'
Row 4814: Found 'chorus' or 'verse'
Row 4821: Found 'chorus' or 'verse'
Row 4834: Found 'chorus' or 'verse'
Row 4838: Found 'chorus' or 'verse'
Row 4856: Found 'chorus' or 'verse'
Row 4862: Found 'chorus' or 'verse'
Row 4865: Found 'chorus' or 'verse'
Row 4867: Found 'chorus' or 'verse'
Row 4876: Found 'chorus' or 'verse'
Row 4879: Found 'chorus' or 'verse'
Row 4880: Found 'chorus' or 'verse'
Row 4883: Found 'chorus' or 'verse'
Row 4884: Found 'chorus' or 'verse'
Row 4892: Found 'chorus' or 'verse'
Row 4894: Found 'chorus' or 'verse'
Row 4900: Found 'chorus' or 'verse'
Row 4910: Found 'chorus' or 'verse'
Row 4914: Found 'chorus' or 'verse'
Row 4947: Found 'chorus' or 'verse'
Row 4949: Found 'chorus' or 

In [43]:
for index, row in lyrics_en.iterrows():
    lyrics_list = row['Lyrics_Clean']
    
    # Remove 'chorus' and 'verse' from the lyrics list
    cleaned_lyrics = [word for word in lyrics_list if word.lower() not in ['chorus', 'verse']]
    
    # Update the 'Lyrics_Clean' column with the cleaned lyrics list
    lyrics_en.at[index, 'Lyrics_Clean'] = cleaned_lyrics

# Display the DataFrame after modifying the 'Lyrics_Clean' column
print(lyrics_en)

                  ALink                                              SName  \
0       /ivete-sangalo/                                   Careless Whisper   
1       /ivete-sangalo/  Could You Be Loved / Citação Musical do Rap: S...   
2       /ivete-sangalo/                             Cruisin' (Part. Saulo)   
3       /ivete-sangalo/                                               Easy   
4       /ivete-sangalo/                  For Your Babies (The Voice cover)   
...                 ...                                                ...   
191809   /clegg-johnny/                                        The Waiting   
191810   /clegg-johnny/                              Too Early For The Sky   
191811   /clegg-johnny/      Warsaw 1943 (I Never Betrayed The Revolution)   
191812   /clegg-johnny/                         When The System Has Fallen   
191813   /clegg-johnny/                                Woman Be My Country   

                                                    SLink  \
0 

In [44]:
lyrics_en

,ALink,SName,SLink,Lyric,language,Lyrics_Clean
0,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en,"[i, feel, so, unsure, as, i, take, your, hand,..."
1,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en,"[dont, let, them, fool, ya, or, even, try, to,..."
2,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en,"[baby, lets, cruise, away, from, here, dont, b..."
3,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en,"[know, it, sounds, funny, but, i, just, cant, ..."
4,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en,"[youve, got, that, look, again, the, one, i, h..."
...,...,...,...,...,...,...
191809,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en,"[here, we, stand, waiting, on, the, plain, dar..."
191810,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en,"[i, nearly, disappeared, into, the, mouth, of,..."
191811,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en,"[amambuka, amambuka, azothengisa, izwe, lakith..."
191812,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en,"[sweat, in, the, heat, for, days, on, end, wai..."


In [45]:
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carolinacraus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
for index, row in lyrics_en.iterrows():
    lyrics_list = row['Lyrics_Clean']
    
    # Remove stop words from the lyrics list
    stop_words = set(stopwords.words('english'))
    cleaned_lyrics = [word for word in lyrics_list if word.lower() not in stop_words]
    
    # Update the 'Lyrics_Clean' column with the cleaned lyrics list
    lyrics_en.at[index, 'Lyrics_Clean'] = cleaned_lyrics

In [48]:
lyrics_en

,ALink,SName,SLink,Lyric,language,Lyrics_Clean
0,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en,"[feel, unsure, take, hand, lead, dance, floor,..."
1,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en,"[dont, let, fool, ya, even, try, school, ya, o..."
2,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en,"[baby, lets, cruise, away, dont, confused, way..."
3,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en,"[know, sounds, funny, cant, stand, pain, girl,..."
4,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en,"[youve, got, look, one, hoped, lad, face, beam..."
...,...,...,...,...,...,...
191809,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en,"[stand, waiting, plain, darkness, taken, last,..."
191810,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en,"[nearly, disappeared, mouth, crocodile, nearly..."
191811,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en,"[amambuka, amambuka, azothengisa, izwe, lakith..."
191812,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en,"[sweat, heat, days, end, waiting, come, hear, ..."


In [1]:
lyrics_en.to_csv('lyrics_clean_2.csv', index=False)

NameError: name 'lyrics_en' is not defined

In [50]:
merged_df = pd.merge(lyrics_en, artists, left_on='ALink', right_on='Link', how='left')
merged_df

,ALink,SName,SLink,Lyric,language,Lyrics_Clean,Artist,Genres,Songs,Popularity,Link
0,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en,"[feel, unsure, take, hand, lead, dance, floor,...",Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en,"[dont, let, fool, ya, even, try, school, ya, o...",Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
2,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en,"[baby, lets, cruise, away, dont, confused, way...",Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
3,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en,"[know, sounds, funny, cant, stand, pain, girl,...",Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
4,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en,"[youve, got, look, one, hoped, lad, face, beam...",Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
...,...,...,...,...,...,...,...,...,...,...,...
191809,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en,"[stand, waiting, plain, darkness, taken, last,...",Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/
191810,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en,"[nearly, disappeared, mouth, crocodile, nearly...",Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/
191811,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en,"[amambuka, amambuka, azothengisa, izwe, lakith...",Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/
191812,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en,"[sweat, heat, days, end, waiting, come, hear, ...",Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/


In [52]:
unique_genres = merged_df['Genres'].unique()

for genre in unique_genres:
    print(genre)

Pop; Axé; Romântico
MPB; Axé
Axé
Axé; Romântico; Trilha Sonora
Axé; Pop; Romântico
Axé; Dance; Pop/Rock
Pop; R&B; Black Music
R&B; Pop; Black Music
Hip Hop; Rap; Black Music
R&B; Black Music; Soul Music
Black Music; Soul Music; R&B
Electronica; Black Music; Hip Hop
Romântico; Trilha Sonora; Black Music
Hip Hop; Dance; Pop
Rap; Black Music; Pop
Rap; Hip Hop; Pop
Hip Hop; R&B; Rap
Rap; Pop; Hip Hop
Hip Hop; Black Music; R&B
Dance; Pop; R&B
Black Music; Soul Music; Hip Hop
Black Music; R&B; Hip Hop
Dance; Black Music; Hip Hop
Black Music; Hip Hop; Rap
Rap; Black Music; Hip Hop
Gospel/Religioso; Pop; Black Music
Hip Hop; Pop; R&B
Pop; R&B; Hip Hop
MPB; Samba; Black Music
R&B; Black Music; Rap
Black Music; Rap; Hip Hop
R&B; Black Music; Funk
Gospel/Religioso; Black Music; Soul Music
Hip Hop; R&B; Black Music
Romântico; Black Music
Soul Music; Pop; Disco
Pop; Dance; Hip Hop
Black Music; Romântico; Pop
Black Music; Reggae; Hip Hop
Hip Hop; Black Music; Rap
R&B; Black Music; Pop
Electronica; D

In [53]:
genre_counts = merged_df['Genres'].value_counts().reset_index()
genre_counts.columns = ['Genre', 'Count']
genre_counts

,Genre,Count
0,Country,4774
1,Rock,4672
2,Heavy Metal,4394
3,Indie,4289
4,Rap; Hip Hop,3426
...,...,...
1185,J-Pop/J-Rock; Rock; Trilha Sonora,1
1186,Surf Music; Reggae; Romântico,1
1187,Pop/Rock; Dance; Romântico,1
1188,J-Pop/J-Rock; Trilha Sonora; Pop/Rock,1


In [54]:
selected_genres = ['Country', 'Rock', 'Indie', 'Rap; Hip Hop']
filtered_df = merged_df[merged_df['Genres'].isin(selected_genres)].reset_index(drop=True)
filtered_df

,ALink,SName,SLink,Lyric,language,Lyrics_Clean,Artist,Genres,Songs,Popularity,Link
0,/alan-jackson/,Livin' On Love,/alan-jackson/livin-on-love.html,Two young people without a thing\nSay some vow...,en,"[two, young, people, without, thing, say, vows...",Alan Jackson,Country,276.0,7.6,/alan-jackson/
1,/alan-jackson/,I'll Try,/alan-jackson/ill-try.html,"Here we are, talkin' 'bout forever\nBoth know ...",en,"[talkin, bout, forever, know, damn, well, easy...",Alan Jackson,Country,276.0,7.6,/alan-jackson/
2,/alan-jackson/,I'll Go On Loving You,/alan-jackson/ill-go-on-loving-you.html,When I look into your soft green eyes\nAnd whe...,en,"[look, soft, green, eyes, see, delicate, body,...",Alan Jackson,Country,276.0,7.6,/alan-jackson/
3,/alan-jackson/,Chattahoochee,/alan-jackson/chattahoochee.html,Way down yonder on the Chattahoochee\nIt gets ...,en,"[way, yonder, chattahoochee, gets, hotter, hoo...",Alan Jackson,Country,276.0,7.6,/alan-jackson/
4,/alan-jackson/,Little Bitty,/alan-jackson/little-bitty.html,Have a little love on a little honeymoon\nYou ...,en,"[little, love, little, honeymoon, got, little,...",Alan Jackson,Country,276.0,7.6,/alan-jackson/
...,...,...,...,...,...,...,...,...,...,...,...
17156,/monks/,Love Can Tame The Wild,/monks/love-can-tame-the-wild.html,Live as one will come a son\nLove's a thing th...,en,"[live, one, come, son, loves, thing, loves, si...",The Monks,Rock,12.0,0.0,/monks/
17157,/monks/,Monk Time,/monks/monk-time.html,"Alright, my name's Gary.\nLet's go, it's beat ...",en,"[alright, names, gary, lets, go, beat, time, h...",The Monks,Rock,12.0,0.0,/monks/
17158,/monks/,Nice Legs Shame About The Face,/monks/nice-legs-shame-about-the-face.html,"Met her on a blind date, helping out an old ma...",en,"[met, blind, date, helping, old, mate, waiting...",The Monks,Rock,12.0,0.0,/monks/
17159,/monks/,Shut Up,/monks/shut-up.html,"Got a reason to laugh,\nGot a reason to cry.\n...",en,"[got, reason, laugh, got, reason, cry, believi...",The Monks,Rock,12.0,0.0,/monks/


In [55]:
filtered_df = filtered_df.drop(['Songs', 'Link'], axis=1)
filtered_df

,ALink,SName,SLink,Lyric,language,Lyrics_Clean,Artist,Genres,Popularity
0,/alan-jackson/,Livin' On Love,/alan-jackson/livin-on-love.html,Two young people without a thing\nSay some vow...,en,"[two, young, people, without, thing, say, vows...",Alan Jackson,Country,7.6
1,/alan-jackson/,I'll Try,/alan-jackson/ill-try.html,"Here we are, talkin' 'bout forever\nBoth know ...",en,"[talkin, bout, forever, know, damn, well, easy...",Alan Jackson,Country,7.6
2,/alan-jackson/,I'll Go On Loving You,/alan-jackson/ill-go-on-loving-you.html,When I look into your soft green eyes\nAnd whe...,en,"[look, soft, green, eyes, see, delicate, body,...",Alan Jackson,Country,7.6
3,/alan-jackson/,Chattahoochee,/alan-jackson/chattahoochee.html,Way down yonder on the Chattahoochee\nIt gets ...,en,"[way, yonder, chattahoochee, gets, hotter, hoo...",Alan Jackson,Country,7.6
4,/alan-jackson/,Little Bitty,/alan-jackson/little-bitty.html,Have a little love on a little honeymoon\nYou ...,en,"[little, love, little, honeymoon, got, little,...",Alan Jackson,Country,7.6
...,...,...,...,...,...,...,...,...,...
17156,/monks/,Love Can Tame The Wild,/monks/love-can-tame-the-wild.html,Live as one will come a son\nLove's a thing th...,en,"[live, one, come, son, loves, thing, loves, si...",The Monks,Rock,0.0
17157,/monks/,Monk Time,/monks/monk-time.html,"Alright, my name's Gary.\nLet's go, it's beat ...",en,"[alright, names, gary, lets, go, beat, time, h...",The Monks,Rock,0.0
17158,/monks/,Nice Legs Shame About The Face,/monks/nice-legs-shame-about-the-face.html,"Met her on a blind date, helping out an old ma...",en,"[met, blind, date, helping, old, mate, waiting...",The Monks,Rock,0.0
17159,/monks/,Shut Up,/monks/shut-up.html,"Got a reason to laugh,\nGot a reason to cry.\n...",en,"[got, reason, laugh, got, reason, cry, believi...",The Monks,Rock,0.0


In [56]:
filtered_df.to_csv('4genres.csv', index=False)